# Valuing Caps and Floors

We value caps and floors using Black's model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.rates import *

####################################################################
# FINANCEPY BETA Version 0.191 - This build:  17 Jan 2021 at 18:30 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Building a Ibor Curve

In [3]:
valuationDate = TuringDate(6, 6, 2018)

In [4]:
spotDays = 0
settlementDate = valuationDate
depoDCCType = TuringDayCountTypes.THIRTY_E_360_ISDA

depos = []
depo = TuringIborDeposit(settlementDate, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "2M", 0.0230, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "3M", 0.0230, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "6M", 0.0230, depoDCCType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "9M", 0.0230, depoDCCType); depos.append(depo)

In [5]:
accrual = TuringDayCountTypes.THIRTY_E_360
freq = TuringFrequencyTypes.SEMI_ANNUAL
longEnd = TuringDateGenRuleTypes.BACKWARD
swapType = TuringSwapTypes.PAY

spotDays = 2
settlementDate = valuationDate.addWeekDays(spotDays)

swaps = []
swap = TuringIborSwap(settlementDate, "1Y", swapType, 0.0250, freq, accrual); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "2Y", swapType, 0.0255, freq, accrual); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "3Y", swapType, 0.0260, freq, accrual); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "4Y", swapType, 0.0265, freq, accrual); swaps.append(swap)
swap = TuringIborSwap(settlementDate, "5Y", swapType, 0.0270, freq, accrual); swaps.append(swap)

In [6]:
liborCurve = TuringIborSingleCurve(valuationDate, depos, [], swaps)

## Creating a Cap and Floor

In [9]:
capType = TuringCapFloorTypes.CAP
floorType = TuringCapFloorTypes.FLOOR

In [10]:
strikeRate = 0.02

In [11]:
cap = TuringIborCapFloor(settlementDate, "2Y", capType, strikeRate)
flr = TuringIborCapFloor(settlementDate, "2Y", floorType, strikeRate)

In [12]:
print(cap)

OBJECT TYPE: TuringIborCapFloor
START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
STRIKE COUPON: 2.0
OPTION TYPE: TuringCapFloorTypes.CAP
FREQUENCY: TuringFrequencyTypes.QUARTERLY
DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA


In [13]:
print(flr)

OBJECT TYPE: TuringIborCapFloor
START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
STRIKE COUPON: 2.0
OPTION TYPE: TuringCapFloorTypes.FLOOR
FREQUENCY: TuringFrequencyTypes.QUARTERLY
DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA


# Valuation

### Black's Model

We start with Black's model with 25% volatility

In [14]:
model = TuringModelBlack(0.25)

In [15]:
cap.value(valuationDate, liborCurve, model)

11364.358596518792

In [16]:
flr.value(valuationDate, liborCurve, model)

830.1408120616613

In [17]:
cap.printLeg()

START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
OPTION TYPE TuringCapFloorTypes.CAP
STRIKE (%): 2.0
FREQUENCY: TuringFrequencyTypes.QUARTERLY
DAY COUNT: TuringDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE 06-JUN-2018
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
    08-JUN-2018          -         -            -     1.000000            -            -
    10-SEP-2018  0.2555556   2.29838       757.99     0.994034       757.99       757.99
    10-DEC-2018  0.2500000   2.28739       710.13     0.988382       760.00      1517.99
    08-MAR-2019  0.2444444   2.29062       698.24     0.982878       815.49      2333.48
    10-JUN-2019  0.2555556   3.08483      2703.56     0.975190      2714.74      5048.22
    09-SEP-2019  0.2472222   2.60810      1456.67     0.968943      1558.15      6606.37
    09-DEC-2019  0.2500000   2.57912      1393.85     0.962735      1544.42      8150.79
    09-MAR-2020  0.2500000   2.57912      1384.92     0.956568    

# Alternative Models

## Shifted Black

Shifted Black gives the same pdf at F+S as Black does at F. So if we want to have the PDF for F=0.25 at -0.25 because rates are negative then you need to set F=-0.50.

In [18]:
model = TuringModelBlackShifted(0.25, -0.005)

In [19]:
cap.value(valuationDate, liborCurve, model)

10931.197435082488

In [20]:
flr.value(valuationDate, liborCurve, model)

396.9796506253582

The floor has increased in value as the downside risk is greater.

## SABR Model

In [21]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573

In [22]:
model = TuringModelSABR(alpha, beta, rho, nu)

In [23]:
cap.value(valuationDate, liborCurve, model)

11467.46038874577

In [24]:
flr.value(valuationDate, liborCurve, model)

933.2426042886418

## Shifted SABR Model

In [25]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573; shift = -0.005

In [26]:
model = TuringModelSABRShifted(alpha, beta, rho, nu, shift)

In [27]:
cap.value(valuationDate, liborCurve, model)

11834.498534976763

In [28]:
flr.value(valuationDate, liborCurve, model)

1300.2807505196336

## Hull White Model

In [29]:
sigma = 0.01; alpha = 0.005

In [30]:
model = TuringModelRatesHW(sigma, alpha)

In [31]:
cap.value(valuationDate, liborCurve, model)

14292.202257995175

In [32]:
flr.value(valuationDate, liborCurve, model)

2773.738790002051

Copyright (c) 2020 Dominic O'Kane